# Deep learning - Kaggle

In [ ]:
# load data
import os
from os.path import join


hot_dog_image_dir = '../input/hot-dog-not-hot-dog/seefood/train/hot_dog'

hot_dog_paths = [join(hot_dog_image_dir,filename) for filename in 
                            ['1000288.jpg',
                             '127117.jpg']]

not_hot_dog_image_dir = '../input/hot-dog-not-hot-dog/seefood/train/not_hot_dog'
not_hot_dog_paths = [join(not_hot_dog_image_dir, filename) for filename in
                            ['823536.jpg',
                             '99890.jpg']]

img_paths = hot_dog_paths + not_hot_dog_paths

In [ ]:
# importing the libraries
from IPython.display import Image, display
from learntools.deep_learning.decode_predictions import decode_predictions
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
# Preparing the data 
image_size = 224

#This function receive the image path and the image size and use the function "load_img" and "img_to_array" to preprocess the data
def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    return(output)

In [ ]:
# Creating the model 

# The model that will be used is ResNet model
# ResNet : model used to train extremely deep neural networks 
my_model = ResNet50(weights='../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels.h5') #pre-defined weights
test_data = read_and_prep_images(img_paths)
preds = my_model.predict(test_data)

most_likely_labels = decode_predictions(preds, top=3)

In [ ]:
# Visualize predictions
for i, img_path in enumerate(img_paths):
    display(Image(img_path))
    print(most_likely_labels[i])

In [ ]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_3 import *
print("Setup Complete")

# EXERCISES

1) We want to distinguish whether an image is a hot dog or not. But our models classify pictures into 1000 different categories. Write a function that takes the models predictions (in the same format as `preds` from the set-up code) and returns a list of `True` and `False` values.

Some tips:
- Work iteratively. Figure out one line at a time outsie the function, and print that line's output to make sure it's right. Once you have all the code you need, move it into the function `is_hot_dog`. If you get an error, check that you have copied the right code and haven't left anything out.
- The raw data we loaded in `img_paths` had two images of hot dogs, followed by two images of other foods. So, if you run your function on `preds`, which represents the output of the model on these images, your function should return `[True, True, False, False]`.
- You will want to use the `decode_predictions` function that was also used in the code provided above. We provided a line with this in the code cell to get you started.

In [ ]:
# Experiment with code outside the function, then move it into the function once you think it is right

# the following lines are given as a hint to get you started
decoded = decode_predictions(preds, top=1)
print(decoded)
import re

def is_hot_dog(preds):
    '''
    inputs:
    preds_array:  array of predictions from pre-trained model
    
    outputs:
    is_hot_dog_list: a list indicating which predictions show hotdog as the most likely label
    '''
    
    preds_array = decode_predictions(preds, top=1)
    is_hot_dog_list = []
    for pred in preds_array:
        for item in pred:
            x = re.search("hotdog",item[1])
            if(x): 
                result =True
            else:
                result = False
            
            is_hot_dog_list.append(result)     
            
        pass 
    return is_hot_dog_list
    
q_1.check()

### Exercise 2: Evaluate Model Accuracy

You have a model (called 'y_model'. Is it good enough to build your app around? 

Find out by writing a function that calculates a model's accuracy (fraction correct). You will try an alternative model in the next step. So we will put this logic in a reusable function that takes data and the model as arguments, and returns the accuracy.

Tips:

 - Use the `is_hot_dog` function from above to help write your function
 - To save you some scrolling, here is the code from above where we used a TensorFlow model to make predictions:

'''
my_model = ResNet50(weights='../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
test_data = read_and_prep_images(img_paths)
preds = my_model.predict(test_data)
'''

In [ ]:
def calc_accuracy(model, paths_to_hotdog_images, paths_to_other_images):
    
    #Preparing the images
    data1 = read_and_prep_images(paths_to_hotdog_images)
    data2 = read_and_prep_images(paths_to_other_images)
    
    #Make the predictions with the defined model
    pred_d1 = model.predict(data1)
    
    pred_d2 = model.predict(data2)
    
    #Analizing if a image is or not a hotdog
    list_d1 = is_hot_dog(pred_d1)
    
    list_d2 = is_hot_dog(pred_d2)
    
    #Computing the accuracy as the fraction between list_d1 and list_d2
    
    size = len(pred_d1)*len(pred_d2)
    r = 0
    for d1 in pred_d1:
        for d2 in pred_d2:
            r +=  d1.any() / d2.any()
        pass  
    
    accu = r / size
    return(accu)

# Code to call calc_accuracy.  my_model, hot_dog_paths and not_hot_dog_paths were created in the setup code
my_model_accuracy = calc_accuracy(my_model, hot_dog_paths, not_hot_dog_paths)
print("Fraction correct in small test set: {}".format(my_model_accuracy))

# checks that your function calc_accuracy works correctly
q_2.check()

### Exercise 3:
There are other models besides the ResNet model (which we have loaded). For example, an earlier winner of the ImageNet competition is the VGG16 model.  Don't worry about the differences between these models yet. We'll come back to that later. For now, just focus on the mechanics of applying these models to a problem.

The code used to load a pretrained ResNet50 model was

```
my_model = ResNet50(weights='../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
```

The weights for the model are stored at `../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5`.

In the cell below, create a VGG16 model with the preloaded weights. Then use your `calc_accuracy` function to determine what fraction of images the VGG16 model correctly classifies.  Is it better or worse than the pretrained ResNet model?

In [ ]:
# import the model
from tensorflow.keras.applications import VGG16


vgg16_model = VGG16(weights='../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5')
# calculate accuracy on small dataset as a test
vgg16_accuracy = calc_accuracy(vgg16_model, hot_dog_paths, not_hot_dog_paths)


print("Fraction correct in small dataset: {}".format(vgg16_accuracy))
q_3.check()

# Intro

You don't directly choose the numbers to go into your convolutions for deep learning... instead the deep learning technique determines what convolutions will be useful from the data (as part of model-training). We'll come back to how the model does that soon.

![Imgur](https://i.imgur.com/op9Maqr.png)

But looking closely at convolutions and how they are applied to your image will improve your intuition for these models, how they work, and how to debug them when they don't work.

**Let's get started.**

# Exercises
We'll use some small utilty functions to visualize raw images and some results of your code. Execute the next cell to load the utility functions.

In [2]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_1 import *
print("Setup Complete")

ModuleNotFoundError: No module named 'learntools'

### Exercise 1

In the video, you saw a convolution that detected horizontal lines. That convolution shows up again in the code cell below.

Run the cell to see a raw image as well as the output from applying this convolution to the image.


In [ ]:
horizontal_line_conv = [[1, 1], 
                        [-1, -1]]
# load_my_image and visualize_conv are utility functions provided for this exercise
original_image = load_my_image() 
visualize_conv(original_image, horizontal_line_conv)

Now it's your turn. Instead of a horizontal line detector, you will create a vertical line detector.

**Replace the underscores with numbers to make a vertical line detector and uncomment both lines of code in the cell below. Then run **

In [ ]:
vertical_line_conv = [[-1,1],[-1,1]]

q_1.check()
visualize_conv(original_image, vertical_line_conv)

### Exercise 2
The convolutions you've seen are 2x2.  But you could have larger convolutions. They could be 3x3, 4x4, etc.  They don't even have to be square. Nothing prevents using a 4x7 convolution.

Compare the number of visual patterns that can be captured by small convolutions. Which of the following is true?

- There are more visual patterns that can be captured by large convolutions
- There are fewer visual patterns that can be captured by large convolutions
- The number of visual patterns that can be captured by large convolutions is the same as the number of visual patterns that can be captured by small convolutions?

Once you think you know the answer, check it by uncommenting and running the line below.

# Transfer - learning

# Exercise Introduction

The cameraman who shot our deep learning videos mentioned a problem that we can solve with deep learning.  

He offers a service that scans photographs to store them digitally.  He uses a machine that quickly scans many photos. But depending on the orientation of the original photo, many images are digitized sideways.  He fixes these manually, looking at each photo to determine which ones to rotate.

In this exercise, you will build a model that distinguishes which photos are sideways and which are upright, so an app could automatically rotate each image if necessary.

If you were going to sell this service commercially, you might use a large dataset to train the model. But you'll have great success with even a small dataset.  You'll work with a small dataset of dog pictures, half of which are rotated sideways.

Specifying and compiling the model look the same as in the example you've seen. But you'll need to make some changes to fit the model.

**Run the following cell to set up automatic feedback.**

In [ ]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_4 import *
print("Setup Complete")

# 1) Specify the Model

Since this is your first time, we'll provide some starter code for you to modify. You will probably copy and modify code the first few times you work on your own projects.

There are some important parts left blank in the following code.

Fill in the blanks (marked with `____`) and run the cell

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Indicate whether the first layer should be trained/changed or not.
my_new_model.layers[0].trainable = False

step_1.check()

# 2) Compile the Model

You now compile the model with the following line.  Run this cell.

In [ ]:
my_new_model.compile(optimizer='sgd', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

# 3) Review the Compile Step
You provided three arguments in the compile step.  
- optimizer
- loss
- metrics

Which arguments could affect the accuracy of the predictions that come out of the model?  After you have your answer, run the cell below to see the solution.

# Solution:

- optimizer determines how we determine the numerical values that make up the model. So it can affect the resulting model and predictions
- loss determines what goal we optimize when determining numerical values in the model. So it can affect the resulting model and predictions
- metrics determines only what we print out while the model is being built, but it doesn't affect the model itself.

You may not understand all of this yet. That's totally fine for now. It will become clearer in an upcoming lesson (called A Deeper Understanding of Deep Learning).

# 4) Fit Model

**Your training data is in the directory `../input/dogs-gone-sideways/images/train`. The validation data is in `../input/dogs-gone-sideways/images/val`**. Use that information when setting up `train_generator` and `validation_generator`.

You have 220 images of training data and 217 of validation data.  For the training generator, we set a batch size of 10. Figure out the appropriate value of `steps_per_epoch` in your `fit_generator` call. #ES DE 22!!! YA QUE 220 imgs en contenedores de 10 nos lleva a 22 steps

Fill in all the blanks (again marked as `____`).  Then run the cell of code.  Watch as your model trains the weights and the accuracy improves.

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224
data_generator = ImageDataGenerator(preprocess_input)

train_generator = data_generator.flow_from_directory(
                                        directory='../input/dogs-gone-sideways/images/train',
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
                                        directory='../input/dogs-gone-sideways/images/val',
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

# fit_stats below saves some statistics describing how model fitting went
# the key role of the following line is how it changes my_new_model by fitting to data
fit_stats = my_new_model.fit_generator(train_generator,
                                       steps_per_epoch=22,
                                       validation_data=validation_generator,
                                       validation_steps=1)

step_4.check()

# Exercise Introduction
We will return to the automatic rotation problem you worked on in the previous exercise. But we'll add data augmentation to improve your model.

The model specification and compilation steps don't change when you start using data augmentation. The code you've already worked with for specifying and compiling a model is in the cell below.  Run it so you'll be ready to work on data augmentation.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_5 import *
print("Setup Complete")

# 1) Fit the Model Using Data Augmentation

Here is some code to set up some ImageDataGenerators. Run it, and then answer the questions below about it.

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224

# Specify the values for all arguments to data_generator_with_aug.
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True,
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)
            
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

Why do we need both a generator with augmentation and a generator without augmentation? After thinking about it, check out the solution below.

Solution: We want to do data augmentation when fitting the model for the reasons mentioned in the video (including a reduction in overfitting, by giving us more data to work with).

But we don't want to change how we test the model. So the validation generator will use an ImageDataGenerator without augmentation. That allows a straightforward comparison between different training procedures (e.g. training with augmentation and without it).

If the augmentation made it harder to predict the label associated with an image (e.g. because of how the image was cropped in augmentation) that would make it misleading to compare scores to another procedure where the validation data was only original images.

# 2) Choosing Augmentation Types
ImageDataGenerator offers many types of data augmentation. For example, one argument is `rotation_range`. This rotates each image by a random amount that can be up to whatever value you specify.

Would it be sensible to use automatic rotation for this problem?  Why or why not?

Solution: The goal in this problem is to tell if an image is upright or sideways. Rotating images moderately might cause images that don't feel cleanly in either category.

Since data augmentation affects images without touching the labels, a dramatic rotation would mean some images are used for training with the wrong label (e.g. an image would be rotated sideways by the generator, and still have a label of being upright)

# 3) Code
Fill in the missing pieces in the following code. We've supplied some boilerplate. You need to think about what ImageDataGenerator is used for each data source.

In [1]:
# Specify which type of ImageDataGenerator above is to load in training data
train_generator = data_generator_with_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/images/train',
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical')

# Specify which type of ImageDataGenerator above is to load in validation data
validation_generator = data_generator_no_aug.flow_from_directory(
        directory = '../input/dogs-gone-sideways/images/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator, # if you don't know what argument goes first, try the hint
        epochs = 3,
        steps_per_epoch=19,
        validation_data=validation_generator)

q_3.check()

228

# 4) Did Data Augmentation Help?
How could you test whether data augmentation improved your model accuracy?

Solution: Create train_generator usng data_generator_no_aug but don't change other arguments to train_generator.

Run the model and see the resuling accuracy. Compare this to the accuracy you got when train_generator used augmentation.

Our validation dataset is very small, so there's a little bit of luck or randomness in the exact score from any model run. Validation scores will be more reliable as you start using larger datasets.

# Introduction

You've seen how to build a model from scratch to identify handwritten digits.  You'll now build a model to identify different types of clothing.  To make models that train quickly, we'll work with very small (low-resolution) images. 

As an example, your model will take an images like this and identify it as a shoe:

![Imgur](https://i.imgur.com/GyXOnSB.png)

# Data Preparation
This code is supplied, and you don't need to change it. Just run the cell below.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

img_rows, img_cols = 28, 28
num_classes = 10

def prep_data(raw):
    y = raw[:, 0]
    out_y = keras.utils.to_categorical(y, num_classes)
    
    x = raw[:,1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x, out_y

fashion_file = "../input/fashionmnist/fashion-mnist_train.csv"
fashion_data = np.loadtxt(fashion_file, skiprows=1, delimiter=',')
x, y = prep_data(fashion_data)

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_7 import *
print("Setup Complete")

# 1) Start the model
Create a `Sequential` model called `fashion_model`. Don't add layers yet.

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D

# Your Code Here
fashion_model = Sequential()
q_1.check()

# 2) Add the first layer

Add the first `Conv2D` layer to `fashion_model`. It should have 12 filters, a kernel_size of 3 and the `relu` activation function. The first layer always requires that you specify the `input_shape`.  We have saved the number of rows and columns to the variables `img_rows` and `img_cols` respectively, so the input shape in this case is `(img_rows, img_cols, 1)`.

In [ ]:
# Your code here
fashion_model.add(Conv2D(12,kernel_size=(3,3),activation='relu',input_shape=(img_rows, img_cols, 1)))
q_2.check()

# 3) Add the remaining layers

1. Add 2 more convolutional (`Conv2D layers`) with 20 filters each, 'relu' activation, and a kernel size of 3. Follow that with a `Flatten` layer, and then a `Dense` layer with 100 neurons. 
2. Add your prediction layer to `fashion_model`.  This is a `Dense` layer.  We alrady have a variable called `num_classes`.  Use this variable when specifying the number of nodes in this layer. The activation should be `softmax` (or you will have problems later).

In [ ]:
# Your code here
fashion_model.add(Conv2D(20,kernel_size=(3,3),activation='relu')) #convolution
fashion_model.add(Conv2D(20,kernel_size=(3,3),activation='relu'))
fashion_model.add(Flatten())                                   #flatten

fashion_model.add(Dense(100,activation='relu'))            #Dense layer
fashion_model.add(Dense(num_classes,activation='softmax')) #prediction layer

q_3.check()

# 4) Compile Your Model
Compile fashion_model with the `compile` method.  Specify the following arguments:
1. `loss = "categorical_crossentropy"`
2. `optimizer = 'adam'`
3. `metrics = ['accuracy']`

In [ ]:
# Your code to compile the model in this cell
fashion_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
q_4.check()

# 5) Fit The Model
Run the command `fashion_model.fit`. The arguments you will use are
1. The data used to fit the model. First comes the data holding the images, and second is the data with the class labels to be predicted. Look at the first code cell (which was supplied to you) where we called `prep_data` to find the variable names for these.
2. `batch_size = 100`
3. `epochs = 4`
4. `validation_split = 0.2` -> 80% for training and 20% for validation

When you run this command, you can watch your model start improving.  You will see validation accuracies after each epoch.

In [2]:
# Your code to fit the model here
fashion_model.fit(x,y, batch_size=100, epochs=4, validation_split=0.2)
q_5.check()

400

# 6) Create A New Model

Create a new model called `second_fashion_model` in the cell below.  Make some changes so it is different than `fashion_model` that you've trained above. The change could be using a different number of layers, different number of convolutions in the layers, etc.

Define the model, compile it and fit it in the cell below.  See how it's validation score compares to that of the original model.

In [ ]:
# Your code below
second_fashion_model = Sequential()

# first layer
second_fashion_model.add(Conv2D(10, kernel_size=(3,3), activation='relu', input_shape=(img_rows, img_cols, 1)))

#Three convolutions
second_fashion_model.add(Conv2D(15, kernel_size=(3,3), activation='relu'))
second_fashion_model.add(Conv2D(15, kernel_size=(3,3), activation='relu'))
second_fashion_model.add(Conv2D(15, kernel_size=(3,3), activation='relu'))

#Flatten
second_fashion_model.add(Flatten())

#Dense layer
second_fashion_model.add(Dense(100,  activation='relu'))

#Prediction layer
second_fashion_model.add(Dense(num_classes, activation='softmax'))

#Compile the model
second_fashion_model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

#Fit the model
second_fashion_model.fit(x,y,batch_size=100, epochs=4, validation_split=0.2)
q_6.check()

# Dropout and Strides for Larger Models

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_8 import *
print("Setup Complete")

img_rows, img_cols = 28, 28
num_classes = 10

def prep_data(raw):
    y = raw[:, 0]
    out_y = keras.utils.to_categorical(y, num_classes)
    
    x = raw[:,1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x, out_y

fashion_file = "../input/fashionmnist/fashion-mnist_train.csv"
fashion_data = np.loadtxt(fashion_file, skiprows=1, delimiter=',')
x, y = prep_data(fashion_data)

ModuleNotFoundError: No module named 'tensorflow'

# 1) Increasing Stride Size in A Layer
Below is a model without strides (or more accurately, with a stride length of 1)

Run it. Notice it's accuracy and how long it takes per epoch. Then you will change the stride length in one of the layers.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout

batch_size = 16
fashion_model_1 = Sequential()
fashion_model_1.add(Conv2D(16, kernel_size=(3, 3),                     #First Conv. layer
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
fashion_model_1.add(Conv2D(16, (3, 3), activation='relu', strides=2))  #Second Conv. layer with a Stride = 2
fashion_model_1.add(Flatten())                                         #Flat layer
fashion_model_1.add(Dense(128, activation='relu'))                     #Dense layer
fashion_model_1.add(Dense(num_classes, activation='softmax'))          #Prediction layer

fashion_model_1.compile(loss=keras.losses.categorical_crossentropy,    #Compile the model
              optimizer='adam',
              metrics=['accuracy'])

fashion_model_1.fit(x, y,                                              #Fit the model
          batch_size=batch_size,
          epochs=3,
          validation_split = 0.2)
q_1.check()

# My new model with Dropout and Strides

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout

batch_size = 16
new_model = Sequential()

#first layer
new_model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
#Dropout
new_model.add(Dropout(0.5))

#Three layers with stride=2
new_model.add(Conv2D(10, kernel_size=(3,3), activation='relu', strides=2))
#Dropout
new_model.add(Dropout(0.5))
new_model.add(Conv2D(10, kernel_size=(3,3), activation='relu', strides=2))
#Dropout
new_model.add(Dropout(0.5))
new_model.add(Conv2D(10, kernel_size=(3,3), activation='relu', strides=2))
#Dropout
new_model.add(Dropout(0.5))

#Flaten
new_model.add(Flatten())

#Dense layer
new_model.add(Dense(128, activation='relu'))

#Prediction layer
new_model.add(Dense(num_classes, activation='softmax'))

#Compile the model
new_model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

#Fit the model

new_model.fit(x,y, batch_size=batch_size, epochs=3, validation_split=0.2)
q_1.check()